In [5]:
from transformers import TrainingArguments, Trainer

In [ ]:
model = BertModel.from_pretrained("./test/saved_model/")


In [ ]:
model = TrainingArguments("BERT_Result/Generated/checkpoint-15000")

In [30]:
from transformers import BertConfig, BertModel, BertForSequenceClassification
# if model is on hugging face Hub
# model = BertModel.from_pretrained("bert-base-uncased")
# from local folder
model = BertForSequenceClassification.from_pretrained("BERT_Result/Generated/checkpoint-15000")

loading configuration file BERT_Result/Generated/checkpoint-15000/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file BERT_Result/Generated/checkpoint-15000/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import datasets
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer


path = "generated_data_mimic.csv"


# Import IMDB reviews
# reviews_df = pd.read_csv(path, encoding="ISO-8859-1", usecols=["review", "label"])
# read in as int
reviews_df = pd.read_csv(
    path, encoding="UTF-8", usecols=["review", "label"], dtype={"review":str,"label": int}
)
# reviews_df = pd.read_csv(path, encoding="UTF-8", usecols=["review", "label"], )
# reviews_df["label"].replace({"neg": 0, "pos": 1}, inplace=True)

reviews_df = reviews_df.dropna().copy()
# Make train and test data
train_set, test_set = train_test_split(reviews_df, test_size=0.2)
print("Train set size: ", train_set.shape[0])
print("Test set size: ", test_set.shape[0])


tds = Dataset.from_pandas(train_set, split="train")
vds = Dataset.from_pandas(test_set, split="validation")

# ds = DatasetDict()
# ds["train"] = tds
# ds["validation"] = vds


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)


# tokenized_datasets = ds.map(tokenize_function, batched=True)

TT_train_set = tds.map(tokenize_function, batched=True)
TT_test_set = vds.map(tokenize_function, batched=True)


loading configuration file config.json from cache at /home/codespace/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/codespace/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab

Train set size:  39997
Test set size:  10000


  0%|          | 0/40 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [13]:
metric = evaluate.load("accuracy")

In [14]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("BERT_Result/Generated_Cauchy/checkpoint-15000")

loading configuration file BERT_Result/Generated_Cauchy/checkpoint-15000/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file BERT_Result/Generated_Cauchy/checkpoint-15000/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassi

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
training_args = TrainingArguments(output_dir="BERT_Result/Generated/")
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
predictions = trainer.predict(TT_test_set)
predictions

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, review. If __index_level_0__, review are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


PredictionOutput(predictions=array([[-0.4658253 ,  0.04038412],
       [-0.46582523,  0.0403812 ],
       [-0.46582526,  0.04038185],
       ...,
       [-0.46582568,  0.04038156],
       [ 0.6526146 , -0.70539355],
       [-0.46582538,  0.04038111]], dtype=float32), label_ids=array([1, 0, 1, ..., 1, 1, 1]), metrics={'test_loss': 0.6276413202285767, 'test_accuracy': 0.661, 'test_runtime': 113.313, 'test_samples_per_second': 88.251, 'test_steps_per_second': 11.031})